# PPO algorithm

In [ ]:
import gym
import numpy as np
import pandas as pd
from gym import spaces
from stable_baselines3 import PPO
import os

# 사용자 정의 환경
class PortfolioEnv(gym.Env):
    def __init__(self, episode_index):
        super(PortfolioEnv, self).__init__()
        # 52-dim state (예: 10개 주식의 일단위 수익률, 63일 평균 수익률, 63일 표준편차, Relative Volume, VIX 지수, 5년 국채금리, previous action)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(52,), dtype=np.float32)
        # 10-dim action: 각 주식의 가중치 (0~1 사이 값, 이후 정규화하여 합=1)
        self.action_space = spaces.Box(low=0, high=1, shape=(10,), dtype=np.float32)
        
        # Load data for the current episode and remove the first column
        raw_data = pd.read_csv(f"data/train_set/{episode_index}.csv").iloc[:, 1:].values
        # Initialize self.data with 42 states from raw_data and append ten 0s for previous action
        self.data = np.hstack((raw_data, np.zeros((raw_data.shape[0], 10))))
        self.max_steps = len(self.data)  # Number of rows in the CSV file
        self.current_step = 0
        self.previous_action = np.zeros(10)  # Initialize previous action as zeros
        self.state = self._next_state()

    def _next_state(self):
        # Use the current row of the CSV file and append previous action as the state
        return np.hstack((self.data[self.current_step, :42], self.previous_action)).astype(np.float32)

    def reset(self):
        self.current_step = 0
        self.previous_action = np.zeros(10)  # Reset previous action to zeros
        self.state = self._next_state()
        return self.state

    def step(self, action):
        # action 정규화: 모든 가중치의 합이 1이 되도록 처리
        weights = action / (np.sum(action) + 1e-8)
        
        # 예시: state의 각 4개 열의 첫 번째 값이 각 주식의 일일 수익률라고 가정
        stock_returns = self.state[:40:4]
        portfolio_return = np.dot(weights, stock_returns)
        
        stock_vols = self.state[2:42:4]
        
        # 위험 패널티: 포트폴리오의 분산을 위험 지표로 사용 (계수는 하이퍼파라미터)
        portfolio_vol = np.sqrt(np.sum((weights * stock_vols) ** 2))
        
        # Calculate turnover as the sum of absolute differences between current and previous actions
        turnover = np.sum(np.abs(weights - self.previous_action))
        
        # Reward: 수익률에서 위험 패널티와 turnover 패널티를 차감 (목표: 수익은 높이고 위험과 turnover는 낮게)
        reward = portfolio_return - 0.1 * portfolio_vol - 0.01 * turnover

        # 다음 state 업데이트 (실제 환경에서는 시장 데이터, 기술적 지표 등으로 전이)
        self.current_step += 1
        if self.current_step < self.max_steps:
            self.state = self._next_state()
        
        # Update previous action
        self.previous_action = weights
        
        done = self.current_step >= self.max_steps
        info = {"portfolio_return": portfolio_return, "risk_penalty": portfolio_vol}
        return self.state, reward, done, info

    def render(self, mode="human"):
        return 1
        # 간단한 출력 예시 (필요 시 구체적으로 구현)
        # print(f"Step: {self.current_step}")

# 학습 코드 (Stable-Baselines3 PPO 사용 예시)
if __name__ == "__main__":
    model_path = "ppo_portfolio"

    # 저장된 모델 파일이 있으면 불러오고, 없으면 새로 학습
    if os.path.exists(model_path + ".zip"):
        model = PPO.load(model_path)
        print("모델을 불러왔습니다:", model_path)
  
    for episode_index in range(10000):  # Using 10000 episodes for training
        # 환경 생성
        env = PortfolioEnv(episode_index)
        model = PPO("MlpPolicy", env, verbose=1)
        model.learn(total_timesteps=env.max_steps)
        # 테스트: 학습된 모델로 환경 실행
        obs = env.reset()
        returns = []
        for i in range(10):
            action, _ = model.predict(obs)
            obs, reward, done, info = env.step(action)
            returns.append(info["portfolio_return"])
            env.render()
            if done:
                break
        
        # Calculate and print Sharpe Ratio
        mean_return = np.mean(returns)
        std_return = np.std(returns)
        sharpe_ratio = mean_return / (std_return + 1e-8)  # Avoid division by zero
        print(f"Episode {episode_index} Sharpe Ratio: {sharpe_ratio}")

        if episode_index % 10 == 9:
            model.save(model_path)
            print("모델 학습 후 저장 완료:", model_path)    
        

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/opt/anaconda3/envs/py311/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 251      |
|    ep_rew_mean     | -3.54    |
| time/              |          |
|    fps             | 8220     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 251         |
|    ep_rew_mean          | -3.53       |
| time/                   |             |
|    fps                  | 4343        |
|    iterations           | 2           |
|    time_elapsed         | 0           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008872864 |
|    clip_fraction        | 0.104       |
|    clip_range           | 0.2         |
|    entropy_loss         | -14.2       |
|    explained_variance   | -3.04       |
|    learning_rate        | 0.

/opt/anaconda3/envs/py311/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 253      |
|    ep_rew_mean     | -3.76    |
| time/              |          |
|    fps             | 8445     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 253          |
|    ep_rew_mean          | -3.68        |
| time/                   |              |
|    fps                  | 5672         |
|    iterations           | 2            |
|    time_elapsed         | 0            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0063565797 |
|    clip_fraction        | 0.0713       |
|    clip_range           | 0.2          |
|    entropy_loss         | -14.2        |
|    explained_variance   | 0.025        |
|    learning_r

...

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 252      |
|    ep_rew_mean     | -3.3     |
| time/              |          |
|    fps             | 8497     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 252          |
|    ep_rew_mean          | -3.38        |
| time/                   |              |
|    fps                  | 5688         |
|    iterations           | 2            |
|    time_elapsed         | 0            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0091886185 |
|    clip_fraction        | 0.0996       |
|    clip_range           | 0.2          |
|    entropy_loss         | -14.2        |
|    explained_variance   | -3.68        |
|    learning_r

/opt/anaconda3/envs/py311/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 252      |
|    ep_rew_mean     | -3.61    |
| time/              |          |
|    fps             | 9008     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 252          |
|    ep_rew_mean          | -3.6         |
| time/                   |              |
|    fps                  | 5966         |
|    iterations           | 2            |
|    time_elapsed         | 0            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0076160887 |
|    clip_fraction        | 0.0862       |
|    clip_range           | 0.2          |
|    entropy_loss         | -14.2        |
|    explained_variance   | -0.346       |
|    learning_r

/opt/anaconda3/envs/py311/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 250      |
|    ep_rew_mean     | -3.4     |
| time/              |          |
|    fps             | 8827     |
|    iterations      | 1        |
|    time_elapsed    | 0        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 250         |
|    ep_rew_mean          | -3.35       |
| time/                   |             |
|    fps                  | 5726        |
|    iterations           | 2           |
|    time_elapsed         | 0           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.007977963 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -14.2       |
|    explained_variance   | -1.6        |
|    learning_rate        | 0.